In [1]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
suppressMessages(library(parallel))

In [2]:
set.seed(42)
getwd()

addArchRThreads(threads = parallel::detectCores() - 2)
addArchRGenome("hg38")

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Cardiomyocyte/code"

Setting default number of Parallel threads to 126.

Setting default genome to Hg38.



In [3]:
df.matching <- readRDS("../data/coembed/ATAC_RNA_matching.rds")
head(df.matching)

,ATAC,RNA,cell_name
,<chr>,<chr>,<chr>
1,CK380#TGCACCTTCAGGGTTT-1,TTTCATGTCGACGTCG-1_1_1_1_1_1_1_1_1_1_1_1_1_1,cell-1
2,CK173#AACCGATCATTGCCTC-1,TTGACCCGTACTGTTG-1_1_1_1_1_1,cell-2
3,CK338#TAACAGCTCCAGGTCG-1,GAAGTAACATGATCTG-1_1,cell-3
4,CK340#CTCAGCTTCGACTTAG-1,GTCAGCGAGACCACGA-1_2_1_1_1_1_1_1_1_1_1_1_1,cell-4
5,CK385#GGGACCTTCTTCATAC-1,CATCGGGAGTCTAGCT-1_2_1_1_1_1_1_1_1_1_1_1_1,cell-5
6,CK340#AACCTTTGTCACAGTT-1,CATTCATCATTGGGAG-1_2_1_1_1_1_1_1_1_1_1,cell-6


In [4]:
coembed <- readRDS("../data/coembed/coembed.annotation.trajectory.Rds")
obj.atac <- coembed[, df.matching$ATAC]

Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”


In [5]:
obj.atac <- obj.atac[, !is.na(obj.atac$healthy_intermediate_damaged)]

Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”
Warning message in Cells(x = value) == Cells(x = x):
“longer object length is not a multiple of shorter object length”


In [6]:
obj.atac

An object of class Seurat 
230075 features across 10689 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 6 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2, dm

In [7]:
## subset atac project
proj <- loadArchRProject("../data/snATAC", showLogo = FALSE)

proj <- subsetArchRProject(proj, 
                           cells = colnames(obj.atac),
                           outputDirectory = "../data/snATAC_healthy_intermediate_damaged",
                           force = TRUE)

Successfully loaded ArchRProject!

Dropping ImputeWeights Since You Are Subsetting Cells! ImputeWeights is a cell-x-cell Matrix!

Copying ArchRProject to new outputDirectory : /data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Cardiomyocyte/data/snATAC_healthy_intermediate_damaged

Copying Arrow Files...

Getting ImputeWeights

No imputeWeights found, returning NULL

Copying Other Files...

Copying Other Files (1 of 14): cell_type_proportion.txt

Copying Other Files (2 of 14): Embeddings

Copying Other Files (3 of 14): GeneScoreMatrix.Rds

Copying Other Files (4 of 14): GroupCoverages

Copying Other Files (5 of 14): IterativeLSI

Copying Other Files (6 of 14): marker_heatmap.Rds

Copying Other Files (7 of 14): MarkerGenes.Rds

Copying Other Files (8 of 14): MarkerGenes.xlsx

Copying Other Files (9 of 14): metadata.csv

Copying Other Files (10 of 14): PeakCalls

Copying Other Files (11 of 14): PeakMatrix.Rds

Copying Other Files (12 of 14): Plots

Copying 

In [8]:
proj


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Cardiomyocyte/data/snATAC_healthy_intermediate_damaged 
samples(19): CK171 CK380 ... CK346 CK170
sampleColData names(1): ArrowFiles
cellColData names(22): Sample TSSEnrichment ... patient_region_id
  patient_group
numberOfCells(1): 10686
medianTSS(1): 8.77
medianFrags(1): 29285.5

In [10]:
## add cell states information
meta.data <- as.data.frame(obj.atac@meta.data)
meta.data <- meta.data[proj@cellColData@rownames, ]
annotation <- meta.data$annotation

proj <- addCellColData(proj, 
                       data = as.character(annotation), 
                        cells = rownames(meta.data),
                       name = "annotation", 
                       force = TRUE)

## add trajectory
trajectory <- meta.data$healthy_intermediate_damaged
proj <- addCellColData(proj, 
                       data = as.numeric(trajectory), 
                        cells = rownames(meta.data),
                       name = "healthy_intermediate_damaged", 
                       force = TRUE)

Overriding previous entry for annotation



In [11]:
## add harmony
embedding <- obj.atac@reductions$harmony@cell.embeddings
embedding <- embedding[rownames(proj), ]

proj@reducedDims[["Harmony"]] <- SimpleList(matDR = as.data.frame(embedding),
                                      params = NULL,
                                           date = Sys.time(),
    scaleDims = NA, #Do not scale dims after
    corToDepth = NA)

In [12]:
## add umap
embedding <- obj.atac@reductions$umap_harmony_v2@cell.embeddings
embedding <- embedding[rownames(proj), ]
colnames(embedding) <- c("Harmony#UMAP_Dimension_1",
                         "Harmony#UMAP_Dimension_2")

proj@embeddings[["umap"]] <- SimpleList(df = as.data.frame(embedding),
                                      params = NULL)

In [13]:
## add dm
embedding <- obj.atac@reductions$dm@cell.embeddings
embedding <- embedding[rownames(proj), ]
colnames(embedding) <- c("Harmony#DM_Dimension_1",
                         "Harmony#DM_Dimension_2")

proj@embeddings[["dm"]] <- SimpleList(df = as.data.frame(embedding),
                                      params = NULL)

In [ ]:
## peak calling for each annotated cell state
pathToMacs2 <- findMacs2()

proj <- addGroupCoverages(ArchRProj = proj, groupBy = "annotation", force = TRUE)

proj <- addReproduciblePeakSet(
    ArchRProj = proj, 
    groupBy = "annotation", 
    pathToMacs2 = pathToMacs2
)

proj <- addPeakMatrix(proj, binarize = TRUE, force = TRUE)

In [ ]:
## add chromVAR
proj <- addMotifAnnotations(ArchRProj = proj, motifSet = "JASPAR2020", name = "Motif",
                           force = TRUE)

proj <- addBgdPeaks(proj, force = TRUE)

proj <- addDeviationsMatrix(
  ArchRProj = proj, 
  peakAnnotation = "Motif",
  force = TRUE,
    binarize = TRUE
)

In [ ]:
## add gene expression
df.matching.sub <- df.matching %>%
    dplyr::filter(ATAC %in% colnames(obj.atac))

obj.rna <- subset(coembed[, df.matching.sub$RNA])

obj.rna

geneMatrix <- getMatrixFromProject(ArchRProj = proj,
                                   useMatrix = "GeneScoreMatrix")

gex.mat <- as.matrix(obj.rna@assays$RNA@counts)
colnames(gex.mat) <- df.matching.sub$ATAC

rowRanges <- rowData(geneMatrix)
sel_genes <- intersect(rownames(gex.mat), rowRanges$name)

length(sel_genes)

gex.mat <- gex.mat[sel_genes, ]
rownames(rowRanges) <- rowRanges$name
rowRanges <- rowRanges[sel_genes, ]

rowRanges <- GRanges(rowRanges$seqnames,
                     IRanges(start = as.numeric(rowRanges$start),
                             end = as.numeric(rowRanges$start) + 1))

seRNA <- SummarizedExperiment(assays = SimpleList(counts = gex.mat),
                              rowRanges = rowRanges)

proj <- addGeneExpressionMatrix(proj,
                                seRNA = seRNA,
                                force = TRUE)

In [ ]:
saveRDS(obj.atac, file = "../data/snATAC_healthy_intermediate_damaged/snATAC.Rds")

saveArchRProject(ArchRProj = proj, 
                 load = FALSE)

In [ ]:
sessionInfo()